# Tambo Filter


We import the necessary libraries 

In [1]:
import pandas as pd
import numpy as np

/home/jd/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


The code and a little explanation is provided 

In [2]:
def Filter_boolean(all_particles_df, charged_particles, length_triangle, a, b):
    """
    Apply geometric and charged particle ID filters to the dataframe of all particles using vectorized operations.
    Adjusted to include only particle ID and time in the output for the provided data structure.
    
    Parameters:
    all_particles_df (pd.DataFrame): Dataframe with all particle data, including 'id', 'x', 'y', and 't' columns.
    charged_particles (list): List of charged particle IDs to filter.
    length_triangle (float): Length of the triangle side.
    a (float): The width of the detector.
    b (float): The height of the detector.
    
    Returns:
    pd.DataFrame: Filtered dataframe with only the ID and time of particles that hit the detectors.
    """
    # Crear una máscara booleana para los IDs de partículas cargadas
    mask_charged = all_particles_df['id'].isin(charged_particles)

    # Calcular las máscaras booleanas para cada detector
    mask_d1 = ((all_particles_df['x'] >= (length_triangle / np.sqrt(3) - b / 2)) & 
               (all_particles_df['x'] <= (length_triangle / np.sqrt(3) + b / 2)) & 
               (all_particles_df['y'] >= -a / 2) & 
               (all_particles_df['y'] <= a / 2))
    
    mask_d2 = ((all_particles_df['x'] >= (-length_triangle / (2 * np.sqrt(3)) - b / 2)) & 
               (all_particles_df['x'] <= (-length_triangle / (2 * np.sqrt(3)) + b / 2)) & 
               (all_particles_df['y'] >= (length_triangle / 2 - a / 2)) & 
               (all_particles_df['y'] <= (length_triangle / 2 + a / 2)))
    
    mask_d3 = ((all_particles_df['x'] >= (-length_triangle / (2 * np.sqrt(3)) - b / 2)) & 
               (all_particles_df['x'] <= (-length_triangle / (2 * np.sqrt(3)) + b / 2)) & 
               (all_particles_df['y'] >= (-length_triangle / 2 - a / 2)) & 
               (all_particles_df['y'] <= (-length_triangle / 2 + a / 2)))

    # Aplicar la máscara de IDs cargados y las máscaras de detectores
    all_particles_df['Detector'] = 0
    all_particles_df.loc[mask_charged & mask_d1, 'Detector'] = 1
    all_particles_df.loc[mask_charged & mask_d2, 'Detector'] = 2
    all_particles_df.loc[mask_charged & mask_d3, 'Detector'] = 3

    # Filtrar las filas donde 'Detector' es diferente de 0 y seleccionar solo las columnas 'id' y 't'
    filtered_df = all_particles_df[all_particles_df['Detector'] != 0][['id', 't', 'Detector']].reset_index(drop=True)
    df_detector_1 = filtered_df[filtered_df['Detector'] == 1].reset_index(drop=True)
    df_detector_2 = filtered_df[filtered_df['Detector'] == 2].reset_index(drop=True)
    df_detector_3 = filtered_df[filtered_df['Detector'] == 3].reset_index(drop=True)

    return df_detector_1, df_detector_2, df_detector_3
